# 1. Import Libraries

In [6]:
import torch

from transformers import (
  T5TokenizerFast as T5Tokenizer,
  T5ForConditionalGeneration
)

# 2. Load Model

In [9]:
OUT_DIR = 'results'
MAX_LENGTH = 1024

In [10]:
model_path = f"{OUT_DIR}/best-checkpoint"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

tokenizer = T5Tokenizer.from_pretrained(OUT_DIR)

# 3. Model Inference

In [11]:
LENGTH_RATIOS = {
  "harsh": (0.10, 0.20),
  "standard": (0.20, 0.35),
  "detailed": (0.30, 0.50),
}

def ensure_complete_sentence(text):
  text = text.strip()
  if text and text[-1] in '.!?':
    return text

  last_period = max(text.rfind('.'), text.rfind('!'), text.rfind('?'))
  if last_period > 0:
    return text[:last_period + 1].strip()

  return text

def summarize_text(text, model, tokenizer, max_length=MAX_LENGTH, summary_style="standard", num_beams=5, no_repeat_ngram_size=3):
  PENALTY_CONFIG = {
    "harsh": {
      "length_penalty": 1.5,
      "repetition_penalty": 1.3
    },
    "standard": {
      "length_penalty": 1.2,
      "repetition_penalty": 1.15
    },
    "detailed": {
      "length_penalty": 0.9,
      "repetition_penalty": 1.1
    }
  }
  
  inputs = tokenizer.encode(
    "summarize: " + text,
    return_tensors='pt',
    max_length=max_length,
    truncation=True
  ).to(device)

  original_word_count = len(text.split())

  min_ratio, max_ratio = LENGTH_RATIOS[summary_style]

  min_length_output = int(original_word_count * min_ratio)
  max_length_output = int(original_word_count * max_ratio)

  max_length_output = max(20, min(max_length_output, int(original_word_count * 0.55)))
  min_length_output = max(5, min(min_length_output, max_length_output - 15))

  length_penalty = PENALTY_CONFIG[summary_style]["length_penalty"]
  repetition_penalty = PENALTY_CONFIG[summary_style]["repetition_penalty"]

  summary_ids = model.generate(
    inputs,
    min_length=min_length_output,
    max_length=max_length_output,
    num_beams=num_beams,
    no_repeat_ngram_size=no_repeat_ngram_size,
    repetition_penalty=repetition_penalty,
    length_penalty=length_penalty,
    early_stopping=True,
    do_sample=False,
    temperature=1.0
  )

  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  summary = ensure_complete_sentence(summary)

  return summary

In [12]:
text = ("""
  Russian President Vladimir Putin said that a US plan to end the war in Ukraine could “form the basis for future agreements” 
  but renewed threats to seize more territory by force unless Kyiv withdraws. Speaking to reporters in Bishtek, in the Central 
  Asian republic of Kyrgyzstan on Thursday, Putin confirmed the Kremlin was expecting a US delegation headed by Special Envoy 
  Steve Witkoff to visit Moscow early next week, adding that the Kremlin was ready for “serious discussion”. But chances of a 
  swift breakthrough appear slim after Putin repeated his maximalist demands, saying the war in Ukraine will only end “once 
  Ukrainian troops withdraw from the territories they occupy.” “If they don’t withdraw, we will achieve this through military 
  means,” the Russian leader said. Russia is occupying some 20% of the territory recognized under international law as part of 
  sovereign Ukraine, including almost all of the Luhansk region, and parts of the Donetsk, Kherson and Zaporizhzhia regions. 
  Moscow is demanding Ukraine surrender the entirety of these four regions, which it has annexed but not fully conquered. Russia 
  has made some gains along the eastern Ukrainian frontline in recent weeks, most significantly around the city of Pokrovsk. Still, 
  the Institute for the Study of War, a US-based conflict monitor, said on Thursday that data on Russian forces’ rate of advance 
  indicates that “a Russian military victory in Ukraine is not inevitable, and a rapid Russian seizure of the rest of Donetsk Oblast 
  is not imminent.” Crucially, the area Russia is demanding includes the “fortress belt” of heavily defended towns and cities seen 
  that are seen as vital for Ukrainian security. Kyiv and its European allies have made it clear that territorial concessions are a red line 
  for them. Putin’s remarks on Thursday were the strongest indication that Russia is not willing to move after US officials, including 
  Trump himself, touted “tremendous progress” being made on their efforts to end the war. This came after Ukrainian and European 
  officials strongly opposed and then revised the 28-point peace plan which was drafted by the US with apparent strong input from Russia. 
  The original plan reflected Russia’s extensive wishlist, and included a demand for Ukraine to reduce its army and be barred from 
  joining NATO. Putin said on Thursday that he was expecting Witkoff to arrive to Moscow early next week, presumably to discuss the 
  new draft of the plan, the exact wording of which is not yet known. Putin said he has been informed of the latest discussions and that 
  the plan could “form the basis for future agreements.” “It would be impolite of me to speak of final agreements now,” Putin added.
""")

print(f"Length of Original Text: {len(text.split())}")
print()

for style in ["harsh", "standard", "detailed"]:
  summary = summarize_text(text, model, tokenizer, summary_style=style)
  print(f"{style.capitalize()} Summary: {summary}")
  print(f"Length of {style.capitalize()} Summary: {len(summary.split())}")
  print()

Length of Original Text: 440

Harsh Summary: Russian President Vladimir Putin has confirmed that he is expecting a US delegation headed by Special Envoy Steve Witkoff to visit Moscow next week. He said that the war in Ukraine will only end once Ukrainian troops withdraw from the territories they occupy.
Length of Harsh Summary: 43

Standard Summary: Russian President Vladimir Putin has confirmed that he is expecting a US delegation headed by Special Envoy Steve Witkoff to visit Moscow next week. He said the war in Ukraine will only end "once Ukrainian troops withdraw from the territories they occupy" Russia has made some gains along the eastern Ukrainian frontline in recent weeks, mostnotably around Pokrovsk. The US has revised the 28-point peace plan drafted by the US.
Length of Standard Summary: 70

Detailed Summary: Russian President Vladimir Putin has confirmed that he is expecting a US delegation headed by Special Envoy Steve Witkoff to visit Moscow next week. He said the war in U